In [1]:
import pandas as pd
from pandas.io import gbq

from sklearn.preprocessing import StandardScaler

In [2]:
pd.set_option('display.float_format', lambda x: '%.2f' % x)

In [3]:
get_data_sql = """
SELECT * FROM `umg-comm-tech-dev.diaspora.india_all_rep_streams`"""

In [ ]:
data = gbq.read_gbq(get_data_sql, project_id='umg-comm-tech-dev', dialect = 'standard')

In [ ]:
data.head()

In [52]:
by_columns = ['stream_country_code','stream_country_name']

In [53]:
data_by_geo = data.groupby(by=by_columns)

In [54]:
df_by_geo = pd.DataFrame(data_by_geo.user_count.sum())
df_by_geo['streams'] = data_by_geo.streams.sum()
df_by_geo = df_by_geo.reset_index()
df_by_geo.head()

,user_dma_number,user_dma_name,user_count,streams
0,,,1125836,2193762
1,500,Portland-Auburn,50,205
2,501,New York,25908,65419
3,502,Binghamton,115,321
4,503,Macon,120,218


In [13]:
get_bench_sql = """
SELECT * FROM `umg-comm-tech-dev.diaspora.benchmark`"""

benchmark = gbq.read_gbq(get_bench_sql, project_id='umg-comm-tech-dev', dialect = 'standard')

In [14]:
benchmark.head()

,user_count,streams,user_dma_number,user_dma_name,user_country_code,user_country_name,stream_country_code,stream_country_name,user_region_code,user_postal_code,user_gender,user_age_group
0,1,23,,,SV,El Salvador,SV,El Salvador,SV-LI,99,female,25-34
1,7631,665437,,,CR,Costa Rica,CR,Costa Rica,CR-SJ,12,male,35-44
2,3,147,,,VN,Vietnam,VN,Vietnam,VN-43,79,male,18-24
3,3,25,,,CR,Costa Rica,CR,Costa Rica,CR-H,61,male,35-44
4,4,109,,,BG,Bulgaria,BG,Bulgaria,BG-04,11,male,25-34


In [55]:
benchmark_by_geo = benchmark.groupby(by=by_columns)

In [56]:
df_benchmark = pd.DataFrame(benchmark_by_geo.user_count.sum())
df_benchmark['streams'] = benchmark_by_geo.streams.sum()
df_benchmark = df_benchmark.reset_index()
df_benchmark.head()

,user_dma_number,user_dma_name,user_count,streams
0,,,329751883,19932797911
1,500,Portland-Auburn,181295,17482670
2,501,New York,6288746,663939045
3,502,Binghamton,53955,5074013
4,503,Macon,71332,4812658


In [57]:
df_benchmark.describe()

,user_count,streams
count,212.00,212.00
mean,2098605.28,139023577.15
std,22648798.08,1370217029.41
min,803.00,67564.00
25%,55539.00,4748639.25
50%,168413.50,12006749.50
75%,447941.50,29008791.00
max,329751883.00,19932797911.00


In [58]:
print(df_benchmark.user_count.sum())
print(df_benchmark.streams.sum())

444904319
29472998355


In [59]:
all_data = pd.merge(df_by_geo, df_benchmark, how='right', on=by_columns, suffixes=['_indian', '_global'])
all_data.head()

,user_dma_number,user_dma_name,user_count_indian,streams_indian,user_count_global,streams_global
0,,,1125836.00,2193762.00,329751883,19932797911
1,500,Portland-Auburn,50.00,205.00,181295,17482670
2,501,New York,25908.00,65419.00,6288746,663939045
3,502,Binghamton,115.00,321.00,53955,5074013
4,503,Macon,120.00,218.00,71332,4812658


In [65]:
all_data_minus_null = all_data.iloc[1:,:]
all_data_minus_null.head()

,user_dma_number,user_dma_name,user_count_indian,streams_indian,user_count_global,streams_global
1,500,Portland-Auburn,50.00,205.00,181295,17482670
2,501,New York,25908.00,65419.00,6288746,663939045
3,502,Binghamton,115.00,321.00,53955,5074013
4,503,Macon,120.00,218.00,71332,4812658
5,504,Philadelphia,5461.00,12026.00,2753208,221478903


In [63]:
to_standardize = ['user_count_indian','streams_indian','user_count_global','streams_global']

In [66]:
# Standardising the variables

scaler = StandardScaler()
data_st=all_data_minus_null.copy()
data_st[to_standardize] = scaler.fit_transform(data_st[to_standardize])
data_st[to_standardize] = pd.DataFrame(data_st[to_standardize],columns=to_standardize)

data_st.head()

/usr/local/lib/python3.7/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/usr/local/lib/python3.7/site-packages/sklearn/base.py:462: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


,user_dma_number,user_dma_name,user_count_indian,streams_indian,user_count_global,streams_global
1,500,Portland-Auburn,-0.22,-0.18,-0.27,-0.25
2,501,New York,6.17,5.39,4.33,5.68
3,502,Binghamton,-0.20,-0.17,-0.37,-0.37
4,503,Macon,-0.20,-0.18,-0.36,-0.37
5,504,Philadelphia,1.12,0.83,1.67,1.62


In [67]:
data_st['users_local_minus_global'] = data_st['user_count_indian']-data_st['user_count_global']
data_st['streams_local_minus_global'] = data_st['streams_indian']-data_st['streams_global']

In [68]:
data_st.head()

,user_dma_number,user_dma_name,user_count_indian,streams_indian,user_count_global,streams_global,users_local_minus_global,streams_local_minus_global
1,500,Portland-Auburn,-0.22,-0.18,-0.27,-0.25,0.05,0.08
2,501,New York,6.17,5.39,4.33,5.68,1.84,-0.29
3,502,Binghamton,-0.20,-0.17,-0.37,-0.37,0.17,0.20
4,503,Macon,-0.20,-0.18,-0.36,-0.37,0.16,0.19
5,504,Philadelphia,1.12,0.83,1.67,1.62,-0.55,-0.79


In [69]:
gbq.to_gbq(data_st, project_id='umg-comm-tech-dev', destination_table='diaspora.indian_minus_bench_std_dma',
          if_exists = 'replace')

1it [00:06,  6.82s/it]


In [70]:
artists = gbq.read_gbq("SELECT * FROM `umg-comm-tech-dev.diaspora.india_artists_genres`",
                      project_id='umg-comm-tech-dev', dialect='standard')

In [71]:
artists.describe(include='all')

,artist_name,genres,followers,popularity,uri
count,2131,2131,2131.00,2131.00,2131
unique,2126,245,nan,nan,2131
top,Sceptre,indian folk,nan,nan,spotify:artist:2JV7rasOwDzJsILh5eQWzA
freq,2,169,nan,nan,1
mean,NaN,NaN,2419.27,13.61,NaN
std,NaN,NaN,17042.00,14.89,NaN
min,NaN,NaN,0.00,0.00,NaN
25%,NaN,NaN,18.00,0.00,NaN
50%,NaN,NaN,124.00,8.00,NaN
75%,NaN,NaN,986.50,25.00,NaN


In [73]:
by_playlist_uri = data.groupby(by='source_uri')

In [79]:
top_playlists = pd.DataFrame(by_playlist_uri.streams.sum().reset_index()).sort_values(by='streams', ascending=False)

In [80]:
gbq.to_gbq(top_playlists, project_id='umg-comm-tech-dev', destination_table='diaspora.playlists_top13',
          if_exists = 'replace')

1it [00:04,  4.57s/it]
